In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
import multibind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '/mnt/c/Users/ignacio.ibarra/Dropbox/annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda:0


In [95]:
adata = mb.bindome.datasets.scATAC.PBMCs_10x_v2(datadir='../../atac_poisson_study/data/')
peak_ids = adata.var_names

In [96]:
# get sequences

In [97]:
adata.var

,gene_ids,feature_types,genome,chr,start,end,peak.length
chr1:9772-10660,chr1:9772-10660,Peaks,GRCh38,chr1,9772,10660,889
chr1:180712-181178,chr1:180712-181178,Peaks,GRCh38,chr1,180712,181178,467
chr1:181200-181607,chr1:181200-181607,Peaks,GRCh38,chr1,181200,181607,408
chr1:191183-192084,chr1:191183-192084,Peaks,GRCh38,chr1,191183,192084,902
chr1:267576-268461,chr1:267576-268461,Peaks,GRCh38,chr1,267576,268461,886
...,...,...,...,...,...,...,...
KI270713.1:13054-13909,KI270713.1:13054-13909,Peaks,GRCh38,KI270713.1,13054,13909,856
KI270713.1:15212-15933,KI270713.1:15212-15933,Peaks,GRCh38,KI270713.1,15212,15933,722
KI270713.1:21459-22358,KI270713.1:21459-22358,Peaks,GRCh38,KI270713.1,21459,22358,900
KI270713.1:29676-30535,KI270713.1:29676-30535,Peaks,GRCh38,KI270713.1,29676,30535,860


In [98]:
seqs = mb.bindome.tl.get_sequences_from_bed(adata.var[['chr', 'start', 'end']], genome='hg38', uppercase=True)
keys = set([s[0] for s in seqs])
adata = adata[:,adata.var_names.isin(keys)]
# seqs = [[s[0], s[1].upper()] for s in seqs[0]]

# remove Ns
seqs = [[s[0], s[1].replace('N', '')] for s in seqs]

counts = adata.X.T

/tmp/tmp9nzixfsr
genome hg38 False
options
/mnt/c/Users/ignacio.ibarra/Dropbox/annotations/hg38/genome/hg38.fa
True /mnt/c/Users/ignacio.ibarra/Dropbox/annotations/hg38/genome/hg38.fa
running bedtools...
bedtools getfasta -fi /mnt/c/Users/ignacio.ibarra/Dropbox/annotations/hg38/genome/hg38.fa -bed /tmp/tmp9nzixfsr -fo /tmp/tmpauti2vvp


WARNING. chromosome (KI270727.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000194.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000194.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000194.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000194.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000194.1) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chromosome (GL000205.2) was not found in the FASTA file. Skipping.
WARNING. chr

In [99]:
next_data = pd.DataFrame.sparse.from_spmatrix(counts)
next_data['seq'] = [s[1] for s in seqs]

/home/rio/miniconda3/envs/multibind/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [100]:
next_data = next_data.head(50000)

In [101]:
# for testing purposes
next_data['seq'] = next_data['seq'].astype(str).str[:min(next_data['seq'].str.len())]

In [102]:
n_rounds = next_data.shape[1] - 2
# n_rounds
dataset = mb.datasets.SelexDataset(next_data, n_rounds=n_rounds)
train = tdata.DataLoader(dataset=dataset, batch_size=256, shuffle=True)

In [103]:
# s = [mb.tl.onehot_mononuc(row['seq'], le, oe) for index, row in next_data.iterrows()]

In [104]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# le = LabelEncoder()
# oe = OneHotEncoder(sparse=False)
# np.array([mb.tl.onehot_mononuc(row['seq'], le, oe) for index, row in next_data.iterrows()])

In [105]:
model = mb.models.DinucSelex(use_dinuc=True, kernels=[0, 14, 8],  n_rounds=n_rounds, enr_series=False).to(device)

Dinuc features not implemented yet. Using only mononuc features.


In [ ]:
optimiser = topti.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = mb.tl.PoissonLoss()
l2 = mb.tl.train_network(model, train, device, optimiser, criterion, num_epochs=200, early_stopping=20, log_each=10)

All TF's, using subsets of only round 0 and 1

In [ ]:
mb.pl.conv_mono(model)

In [4]:
# %time np.array([mb.tl.onehot_mononuc(row['seq'], le, oe) for index, row in next_data.head(10000).iterrows()]).shape

In [5]:
# %time next_data.head(10000)['seq'].apply(mb.tl.onehot_mononuc, args=(le, oe)) # (row['seq'], le, oe) for index, row in next_data.head(10).iterrows()])